In [1]:
# mount the drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import joblib

# Load Model & Scaler

transport_model = joblib.load(
    "/content/drive/MyDrive/Coffee/Dev/Phase 1/C3/transport/transport_demand_model.pkl"
)

transport_scaler = joblib.load(
    "/content/drive/MyDrive/Coffee/Dev/Phase 1/C3/transport/transport_feature_scaler.pkl"
)

In [3]:
# User-level Inputs
area_ha = 1.5
predicted_yield_kg_per_ha = 620
temp = 27.5
feelslike = 29.0
humidity = 78
precip = 18.0
severerisk = 0
month = 7

# Regime shift logic (same as training)
regime_shift = 1 if month >= 1 and 2025 <= 2026 else 0


In [4]:
# Internal Feature Engineering
productivity_index = (
    1.0
    - (precip / 100) * 0.3
    - (humidity / 100) * 0.2
)

productivity_index = np.clip(productivity_index, 0.4, 1.0)


In [5]:
# Build inference DataFrame
df_input = pd.DataFrame([{
    "area_ha": area_ha,
    "predicted_yield_kg_per_ha": predicted_yield_kg_per_ha,
    "temp": temp,
    "feelslike": feelslike,
    "humidity": humidity,
    "precip": precip,
    "severerisk": severerisk,
    "productivity_index": productivity_index,
    "month": month,
    "regime_shift": regime_shift
}])

In [6]:
# FORCE FEATURE ORDER
FEATURE_ORDER = [
    "area_ha",
    "predicted_yield_kg_per_ha",
    "temp",
    "feelslike",
    "humidity",
    "precip",
    "severerisk",
    "productivity_index",
    "month",
    "regime_shift"
]

df_input = df_input[FEATURE_ORDER]

In [7]:
# scaling
X_scaled = transport_scaler.transform(df_input)

In [8]:
# Prediction
vehicles = transport_model.predict(X_scaled)[0]

tractors = int(round(vehicles[0]))
apes = int(round(vehicles[1]))
trucks = int(round(vehicles[2]))

print("Predicted Transportation Demand")
print("--------------------------------")
print(f"Tractors needed : {tractors}")
print(f"APEs needed     : {apes}")
print(f"Trucks needed   : {trucks}")

Predicted Transportation Demand
--------------------------------
Tractors needed : 1
APEs needed     : 0
Trucks needed   : 1
